installez tf2onnx le package pour la conversion 

In [ ]:
!pip install tf2onnx

In [ ]:
from google.colab import drive
import pandas as pd
from sklearn.preprocessing import StandardScaler
import joblib
drive.mount('/content/drive')
# Charger le dataset complet normalis&
df_train = pd.read_csv('/content/drive/MyDrive/train_normalized.csv')
df_val = pd.read_csv('/content/drive/MyDrive/val_normalized.csv')
df_test = pd.read_csv('/content/drive/MyDrive/test_normalized.csv')
df_train.info()
df_train.describe()
df_train.head()

df_val.info()
df_val.describe()
df_val.head()

df_test.info()
df_test.describe()
df_test.head()

In [ ]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from google.colab import drive
from google.colab import files
import joblib
drive.mount('/content/drive')
# Charger les fichiers déjà normalisés
df_train = pd.read_csv('/content/drive/MyDrive/train_normalized.csv')
df_val = pd.read_csv('/content/drive/MyDrive/val_normalized.csv')

# Séparation des features (X) et de la cible (y) pour les deux ensembles
X_train = df_train.drop('Risk_Category', axis=1)
y_train = df_train['Risk_Category']
X_val = df_val.drop('Risk_Category', axis=1)
y_val = df_val['Risk_Category']

# Définir l'architecture du modèle
model = tf.keras.Sequential([
    tf.keras.Input(shape=(14,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Couche de sortie
])

# Compiler le modèle
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Afficher le résumé du modèle
model.summary()

# Définir le chemin de sauvegarde du modèle
checkpoint_path = "/content/drive/MyDrive/IAPFE/model1_checkpoint.keras"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_best_only=True,
    save_weights_only=False,
    monitor='val_accuracy',  # maintenant on surveille val_accuracy !
    mode='max',
    verbose=1
)

# Entraîner le modèle en incluant la validation
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_val, y_val),  # Utilisation du fichier de validation séparé
    callbacks=[model_checkpoint]
)

# Charger le meilleur modèle sauvegardé
model = tf.keras.models.load_model(checkpoint_path)

# Tracer la courbe d'apprentissage
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
epochs = range(1, len(train_accuracy) + 1)

plt.figure(figsize=(10, 6))
plt.plot(epochs, train_accuracy, 'r-', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'b-', label='Validation Accuracy')
plt.title('Courbe d\'apprentissage')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Tracer la courbe de perte (loss)
train_loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(train_loss) + 1)

plt.figure(figsize=(10, 6))
plt.plot(epochs, train_loss, 'r-', label='Training Loss')
plt.plot(epochs, val_loss, 'b-', label='Validation Loss')
plt.title('Courbe de perte (Loss)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()



In [ ]:
# CODE TRES PRESIEUX

import tensorflow as tf
import tf2onnx
from google.colab import drive

# Monter Google Drive
drive.mount('/content/drive')

# Charger le modèle Keras Sequential
seq_model = tf.keras.models.load_model('/content/drive/MyDrive/IAPFE/model1_checkpoint.keras')

# Créer un modèle fonctionnel pour éviter l'erreur .output_names
inputs = tf.keras.Input(shape=(14,), name="input")  # adapte selon ton dataset
outputs = seq_model(inputs)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Conversion vers ONNX
spec = (tf.TensorSpec([None, 14], tf.float32, name="input"),)

onnx_model, _ = tf2onnx.convert.from_keras(
    model,
    input_signature=spec,
    opset=13
)

# Sauvegarde
with open("/content/drive/MyDrive/IAPFE/IOThealth1.0.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

print("✅ Modèle converti avec succès en ONNX !")


In [ ]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from google.colab import drive
from google.colab import files
import joblib
drive.mount('/content/drive')
# Charger les fichiers déjà normalisés
df_train = pd.read_csv('/content/drive/MyDrive/train_normalized.csv')
df_val = pd.read_csv('/content/drive/MyDrive/val_normalized.csv')

# Séparation des features (X) et de la cible (y) pour les deux ensembles
X_train = df_train.drop('Risk_Category', axis=1)
y_train = df_train['Risk_Category']
X_val = df_val.drop('Risk_Category', axis=1)
y_val = df_val['Risk_Category']

# Définir l'architecture du modèle
model = tf.keras.Sequential([
    tf.keras.Input(shape=(14,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Couche de sortie
])

# Compiler le modèle
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Afficher le résumé du modèle
model.summary()

# Définir le chemin de sauvegarde du modèle
checkpoint_path = "/content/drive/MyDrive/IAPFE/model2_checkpoint.keras"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_best_only=True,
    save_weights_only=False,
    monitor='val_accuracy',  # maintenant on surveille val_accuracy !
    mode='max',
    verbose=1
)

# Entraîner le modèle en incluant la validation
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_val, y_val),  # Utilisation du fichier de validation séparé
    callbacks=[model_checkpoint]
)

# Charger le meilleur modèle sauvegardé
model = tf.keras.models.load_model(checkpoint_path)

# Tracer la courbe d'apprentissage
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
epochs = range(1, len(train_accuracy) + 1)

plt.figure(figsize=(10, 6))
plt.plot(epochs, train_accuracy, 'r-', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'b-', label='Validation Accuracy')
plt.title('Courbe d\'apprentissage')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Tracer la courbe de perte (loss)
train_loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(train_loss) + 1)

plt.figure(figsize=(10, 6))
plt.plot(epochs, train_loss, 'r-', label='Training Loss')
plt.plot(epochs, val_loss, 'b-', label='Validation Loss')
plt.title('Courbe de perte (Loss)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# CODE TRES PRESIEUX

import tensorflow as tf
import tf2onnx
from google.colab import drive

# Monter Google Drive
drive.mount('/content/drive')

# Charger le modèle Keras Sequential
seq_model = tf.keras.models.load_model('/content/drive/MyDrive/IAPFE/model2_checkpoint.keras')

# Créer un modèle fonctionnel pour éviter l'erreur .output_names
inputs = tf.keras.Input(shape=(14,), name="input")  # adapte selon ton dataset
outputs = seq_model(inputs)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Conversion vers ONNX
spec = (tf.TensorSpec([None, 14], tf.float32, name="input"),)

onnx_model, _ = tf2onnx.convert.from_keras(
    model,
    input_signature=spec,
    opset=13
)

# Sauvegarde
with open("/content/drive/MyDrive/IAPFE/IOThealth2.0.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

print("✅ Modèle converti avec succès en ONNX !")

In [ ]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from google.colab import drive
from google.colab import files
import joblib
drive.mount('/content/drive')
# Charger les fichiers déjà normalisés
df_train = pd.read_csv('/content/drive/MyDrive/train_normalized.csv')
df_val = pd.read_csv('/content/drive/MyDrive/val_normalized.csv')

# Séparation des features (X) et de la cible (y) pour les deux ensembles
X_train = df_train.drop('Risk_Category', axis=1)
y_train = df_train['Risk_Category']
X_val = df_val.drop('Risk_Category', axis=1)
y_val = df_val['Risk_Category']

# Définir l'architecture du modèle
model = tf.keras.Sequential([
    tf.keras.Input(shape=(14,)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.27),
    tf.keras.layers.Dense(12, activation='relu'),
    tf.keras.layers.Dropout(0.27),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Couche de sortie
])

# Compiler le modèle
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Afficher le résumé du modèle
model.summary()

# Définir le chemin de sauvegarde du modèle
checkpoint_path = "/content/drive/MyDrive/IAPFE/model3_checkpoint.keras"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_best_only=True,
    save_weights_only=False,
    monitor='val_accuracy',  # maintenant on surveille val_accuracy !
    mode='max',
    verbose=1
)

# Entraîner le modèle en incluant la validation
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_val, y_val),  # Utilisation du fichier de validation séparé
    callbacks=[model_checkpoint]
)

# Charger le meilleur modèle sauvegardé
model = tf.keras.models.load_model(checkpoint_path)

# Tracer la courbe d'apprentissage
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
epochs = range(1, len(train_accuracy) + 1)

plt.figure(figsize=(10, 6))
plt.plot(epochs, train_accuracy, 'r-', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'b-', label='Validation Accuracy')
plt.title('Courbe d\'apprentissage')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Tracer la courbe de perte (loss)
train_loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(train_loss) + 1)

plt.figure(figsize=(10, 6))
plt.plot(epochs, train_loss, 'r-', label='Training Loss')
plt.plot(epochs, val_loss, 'b-', label='Validation Loss')
plt.title('Courbe de perte (Loss)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# CODE TRES PRESIEUX

import tensorflow as tf
import tf2onnx
from google.colab import drive

# Monter Google Drive
drive.mount('/content/drive')

# Charger le modèle Keras Sequential
seq_model = tf.keras.models.load_model('/content/drive/MyDrive/IAPFE/model3_checkpoint.keras')

# Créer un modèle fonctionnel pour éviter l'erreur .output_names
inputs = tf.keras.Input(shape=(14,), name="input")  # adapte selon ton dataset
outputs = seq_model(inputs)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Conversion vers ONNX
spec = (tf.TensorSpec([None, 14], tf.float32, name="input"),)

onnx_model, _ = tf2onnx.convert.from_keras(
    model,
    input_signature=spec,
    opset=13
)

# Sauvegarde
with open("/content/drive/MyDrive/IAPFE/IOThealth3.0.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

print("✅ Modèle converti avec succès en ONNX !")